In [1]:
import torch
import matplotlib.pyplot as plt
from itertools import product

In [2]:
class Ising:
    
    # 초기화
    def __init__(self, num_row, num_col):
        self.num_row, self.num_col = num_row, num_col
        self.gen_system()
        
    # (r x s) System configuration을 랜덤으로 생성    
    def gen_system(self, p=0.5):
        self.spins = torch.where(torch.rand(self.num_row, self.num_col) < p, torch.ones(1), -torch.ones(1))
    
    # System의 Hamiltonian를 계산
    def hamiltonian(self):
        H, J = 0., 1.
        for i, j in product(range(self.num_row), range(self.num_col)):
            for i_nhb, j_nhb in self.get_neighbors(i, j):
                H -= J*self.spins[i_nhb, j_nhb]*self.spins[i, j]
        return H/2
            
    # 격자 (i, j)의 neighbors를 리턴
    def get_neighbors(self, i, j):
        assert (i >= 0) and (i <= self.num_row-1) and (j >= 0) and (j <= self.num_col-1)
        nhb = []
        if j != self.num_col - 1: nhb.append([i, j+1])
        if j != 0: nhb.append([i, j-1])
        if i != self.num_row - 1: nhb.append([i+1, j])
        if i != 0: nhb.append([i-1, j])
        return nhb
    
    # n개의 데이터 생성 (Hamiltonian이 낮을수록 생성 확률 높음)
    def gen_train_data(self, num_data):
        self.gen_system()
        H = self.hamiltonian()
        data = ((1-self.spins.view(-1))/2).unsqueeze(0)
        for _ in range(num_data-1):
            self.gen_system()
            H_new = self.hamiltonian()
            p = torch.clamp(torch.exp(-(H_new-H)), max=1)
            if torch.rand(1) < p:
                v = ((1-self.spins.view(-1))/2).unsqueeze(0)
                H = H_new
            else:
                v = data[-1].unsqueeze(0)  
            data = torch.cat([data, v])
        return data

In [20]:
class RBM:
    
    def __init__(self, nv, nh):
        self.nv, self.nh = nv, nh
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(nh)     # nh : Hidden layer 개수 (bias)
        self.b = torch.randn(nv)     # nv : Input data 차원 (?)
        
    def set_train_data(self, train_data):
        self.train_data = train_data
        self.num_data = train_data.shape[0]
        
    def free_energy(self, v):
        bv = torch.dot(self.b, v)
        Wv = torch.mv(self.W, v)
        F = -bv
        for i in range(self.nh):
            F -= torch.log(1+torch.exp(self.a[i] + Wv[i]))
        return F
    
    def loglik(self):
        F = torch.zeros(self.num_data)
        for i in range(self.num_data):
            F[i] = -self.free_energy(self.train_data[i])
        ll = -F.log_softmax(0).mean()
        return ll
    
    def p_v(self):
        F = torch.zeros(self.num_data)
        for i in range(self.num_data):
            F[i] = -self.free_energy(self.train_data[i])
        pv = F.softmax(0)
        return pv
    
    def sigmoid_i(self, idx):
        return torch.sigmoid(self.a + torch.mv(self.W, self.train_data[idx]))
    
    def gradF_i_W(self, idx):
        grad = torch.zeros_like(self.W)
        for i, j in product(range(self.nh), range(self.nv)):
            grad[i, j] -= self.sigmoid_i(idx)[i] * self.train_data[idx][j]
        return grad

    def gradF_i_a(self, idx):
        grad = torch.zeros_like(self.a)
        for i in range(self.nh):
            grad[i] = -self.sigmoid_i(idx)[i]
        return grad

    def gradF_i_b(self, idx):
        grad = torch.zeros_like(self.b)
        for i in range(self.nv):
            grad[i] = -self.train_data[idx][i]
        return grad
    
    def grad_loglik(self, param, gradient):
        grad_loglik = torch.zeros_like(param)
        for idx in range(self.num_data):
            grad_loglik = (1/self.num_data - self.p_v()[idx])*gradient(idx)
        return grad_loglik
    
    def update(self, learning_rate=0.01):
        self.W -= learning_rate*self.grad_loglik(self.W, self.gradF_i_W)
        self.a -= learning_rate*self.grad_loglik(self.a, self.gradF_i_a)
        self.b -= learning_rate*self.grad_loglik(self.b, self.gradF_i_b)

In [25]:
num_row, num_col = 3, 3
ising = Ising(num_row, num_col)
rbm = RBM(num_row*num_col, 2)
rbm.set_train_data(ising.gen_train_data(50))

In [26]:
num_epoch = 10
for epoch in range(num_epoch+1):
    rbm.update(0.1)
    print('epoch {}: W = {}'.format(epoch, rbm.W))
    print('\t a = {}'.format(rbm.a))
    print('\t b = {}'.format(rbm.b))
    print('\t loglik = {}'.format(rbm.loglik()))

epoch 0: W = tensor([[ 0.4080,  0.2312, -0.0856, -0.7043,  0.4428,  0.5958,  0.6631,  0.3955,
         -0.6120],
        [-1.4012,  0.5566,  0.0766, -1.8641,  0.7720, -0.6444, -0.8886,  0.3489,
          0.3515]])
	 a = tensor([-0.3753,  0.5275])
	 b = tensor([-0.3838,  0.5678,  0.7601, -0.5063, -0.4074,  0.6917, -1.5545,  1.1055,
         0.6299])
	 loglik = 4.725319862365723
epoch 1: W = tensor([[ 0.4080,  0.2312, -0.0852, -0.7043,  0.4428,  0.5958,  0.6631,  0.3955,
         -0.6120],
        [-1.4012,  0.5566,  0.0771, -1.8641,  0.7720, -0.6444, -0.8886,  0.3489,
          0.3515]])
	 a = tensor([-0.3750,  0.5280])
	 b = tensor([-0.3838,  0.5678,  0.7609, -0.5063, -0.4074,  0.6917, -1.5545,  1.1055,
         0.6299])
	 loglik = 4.7258148193359375
epoch 2: W = tensor([[ 0.4080,  0.2312, -0.0849, -0.7043,  0.4428,  0.5958,  0.6631,  0.3955,
         -0.6120],
        [-1.4012,  0.5566,  0.0776, -1.8641,  0.7720, -0.6444, -0.8886,  0.3489,
          0.3515]])
	 a = tensor([-0.3747,  0